In [1]:
# Importamos librerías
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, MinMaxScaler,  OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from joblib import dump

In [10]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Cargamos los datos
Data = pd.read_csv("../Datos/data_adults.csv")

Data_cop = Data.drop("fnlwgt", axis=1)
Data_cop = Data_cop.drop("education-num", axis=1)

X = Data_cop.drop("income", axis=1)
y = Data_cop['income'].isin(['>50K.','>50K'])

# Definimos las variables categóricas y numéricas
categorical_features = X.select_dtypes(include=['object']).columns
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns

# Creamos pipelines de preprocesamiento
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Usamos ColumnTransformer para combinar ambas transformaciones
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [11]:
model_pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

In [12]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline

# Dividimos los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creamos un nuevo pipeline que incluye el preprocesamiento y el modelo Naive Bayes
model_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                 ('classifier', GaussianNB())])

# Entrenamos el pipeline completo en los datos de entrenamiento
model_pipeline.fit(X_train, y_train)

# Hacemos predicciones en el conjunto de prueba
y_pred = model_pipeline.predict(X_test)

# Evaluamos el rendimiento del modelo
print(classification_report(y_test, y_pred))



              precision    recall  f1-score   support

       False       0.96      0.55      0.70      7414
        True       0.40      0.92      0.55      2355

    accuracy                           0.64      9769
   macro avg       0.68      0.74      0.63      9769
weighted avg       0.82      0.64      0.67      9769



In [22]:
# Guardamos el pipeline completo (preprocesamiento + modelo entrenado)
dump(model_pipeline, 'model_pipeline_NB.joblib')

['model_pipeline_NB.joblib']

## Optimización de parámetros

In [16]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Cargamos los datos
Data = pd.read_csv("../Datos/data_adults.csv")

Data_cop = Data.drop("fnlwgt", axis=1)
Data_cop = Data_cop.drop("education-num", axis=1)

X = Data_cop.drop("income", axis=1)
y = Data_cop['income'].isin(['>50K.','>50K'])

# Definimos las variables categóricas y numéricas
categorical_features = X.select_dtypes(include=['object']).columns
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns

# Creamos pipelines de preprocesamiento
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Usamos ColumnTransformer para combinar ambas transformaciones
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])




In [23]:

model_nb = GaussianNB()
param_grid = {
}

##Instanciar la busqueda del mejor modelo
model_busqueda = GridSearchCV(estimator=model_nb, 
                              param_grid=param_grid, 
                              cv=3, 
                              verbose=4,
                              scoring='f1',
                              n_jobs=-1)


model_pipeline_train = Pipeline(steps=[('preprocessor', preprocessor),
                                       ('busqueda', model_busqueda)])  


model_pipeline_train.fit(X_train, y_train)



Fitting 3 folds for each of 1 candidates, totalling 3 fits


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['age', 'capital-gain', 'capital-loss', 'hours-per-week'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country'],
      dtype='object'))])),
                ('busqueda',
                 GridSearchCV(cv=3, estimator=GaussianNB(), n_jobs=-1,
                              param_grid={}, scoring='f1', verbose=4))])

In [24]:
busqueda_resultado = model_pipeline_train['busqueda']
busqueda_resultado.best_estimator_.get_params()

{'priors': None, 'var_smoothing': 1e-09}

In [25]:
y_pred = model_pipeline_train.predict(X_test)

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       False       0.96      0.55      0.70      7414
        True       0.40      0.92      0.55      2355

    accuracy                           0.64      9769
   macro avg       0.68      0.74      0.63      9769
weighted avg       0.82      0.64      0.67      9769



In [26]:
y_pred_train = model_pipeline_train.predict(X_train)

print(classification_report(y_train,y_pred_train))

              precision    recall  f1-score   support

       False       0.95      0.55      0.70     29741
        True       0.39      0.91      0.55      9332

    accuracy                           0.64     39073
   macro avg       0.67      0.73      0.62     39073
weighted avg       0.82      0.64      0.66     39073

